In [3]:
!pip install sentence-transformers faiss-cpu numpy


In [8]:
!pip install sentence-transformers transformers torch faiss-cpu pymupdf


In [9]:
import os
import faiss
import pickle
import numpy as np
import torch
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# === CONFIG ===
DATA_FOLDER = "/content/drive/MyDrive/RAG pdf/Financial Report - IBM.pdf"
CHUNK_SIZE = 400
CHUNK_OVERLAP = 50
TOP_K = 3

# === TEXT CHUNKING ===
def chunk_text(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# === LOAD FILES (.txt + .pdf) ===
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return " ".join([page.get_text() for page in doc])

def load_documents(folder):
    documents = []
    if not os.path.exists(folder):
        raise FileNotFoundError(f"Folder '{folder}' not found. Please add files.")

    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filename.endswith(".txt"):
            with open(filepath, "r", encoding="utf-8") as f:
                text = f.read()
        elif filename.endswith(".pdf"):
            text = extract_text_from_pdf(filepath)
        else:
            continue  # skip unsupported

        if text.strip():
            chunks = chunk_text(text)
            for chunk in chunks:
                documents.append({"text": chunk, "source": filename})
    return documents

# === BUILD FAISS INDEX ===
def build_faiss_index(documents, model):
    texts = [doc["text"] for doc in documents]
    embeddings = model.encode(texts)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings).astype("float32"))
    return index, np.array(embeddings), documents

# === RETRIEVE CHUNKS ===
def retrieve_top_k(index, model, query, documents, top_k=TOP_K):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding).astype("float32"), top_k)
    return [documents[i] for i in I[0]]

# === QA MODEL ===
def generate_answer(chunks, query):
    context = " ".join([chunk["text"] for chunk in chunks])
    qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
    result = qa_model(question=query, context=context)
    return result["answer"], context

# === SOURCE ATTRIBUTION ===
def get_sources(chunks):
    return list(set([chunk["source"] for chunk in chunks]))

# === MAIN RAG PIPELINE ===
def rag_pipeline(query):
    print(f"\n🔎 Question: {query}")
    documents = load_documents(DATA_FOLDER)
    if not documents:
        raise ValueError("No valid documents found in the folder.")

    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    index, embeddings, docs = build_faiss_index(documents, embedding_model)
    top_chunks = retrieve_top_k(index, embedding_model, query, docs)
    answer, context = generate_answer(top_chunks, query)
    sources = get_sources(top_chunks)

    print(f"\n✅ Answer: {answer}")
    print("📚 Sources:")
    for src in sources:
        print(f"- {src}")

# === TEST ===
if __name__ == "__main__":
    test_questions = [
        "What are the key components of a Transformer model?",
        "How does positional encoding work in Transformers?",
        "Explain few-shot learning with an example."
    ]

    for question in test_questions:
        rag_pipeline(question)



🔎 Question: What are the key components of a Transformer model?


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/RAG pdf/Financial Report - IBM.pdf'

In [13]:
# Install required libraries (run once in terminal):
# pip install sentence-transformers PyPDF2 scikit-learn transformers torch
!pip install PyPDF2
import os
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# ---------------------------
# Step 1: Document Preprocessing
# ---------------------------

def load_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def vectorize_chunks(chunks, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, convert_to_numpy=True)
    return embeddings, chunks

# ---------------------------
# Step 2: Retriever
# ---------------------------

def retrieve_relevant_chunks(query, embeddings, chunks, model_name='all-MiniLM-L6-v2', top_k=3):
    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query], convert_to_numpy=True)
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [(chunks[i], similarities[i]) for i in top_indices]

# ---------------------------
# Step 3: Answer Generation
# ---------------------------

def generate_answer(query, relevant_chunks, model_name="distilgpt2"):
    context = "\n".join([chunk for chunk, _ in relevant_chunks])
    generator = pipeline("text-generation", model=model_name)
    prompt = f"Question: {query}\nContext: {context}\nAnswer:"
    result = generator(prompt, max_length=200, num_return_sequences=1)[0]['generated_text']
    return result.strip(), context

# ---------------------------
# Step 4: Source Attribution
# ---------------------------

def source_attribution(relevant_chunks):
    return [
        f"Source {i+1}: \"{chunk[:100]}...\" (Similarity: {score:.4f})"
        for i, (chunk, score) in enumerate(relevant_chunks)
    ]

# ---------------------------
# Step 5: Main Execution
# ---------------------------

def run_rag_system(query, pdf_paths):
    all_chunks, all_embeddings = [], []

    for path in pdf_paths:
        print(f"Processing: {path}")
        text = load_pdf(path)
        chunks = chunk_text(text)
        embeddings, _ = vectorize_chunks(chunks)
        all_chunks.extend(chunks)
        all_embeddings.append(embeddings)

    all_embeddings = np.vstack(all_embeddings)
    relevant_chunks = retrieve_relevant_chunks(query, all_embeddings, all_chunks)
    answer, context = generate_answer(query, relevant_chunks)
    citations = source_attribution(relevant_chunks)

    return answer, citations

# ---------------------------
# Example Usage
# ---------------------------

if __name__ == "__main__":
    # 🔺 Make sure your PDF files are in the 'pdfs/' folder
    pdf_folder = "/content/drive/MyDrive/RAG Project Dataset pdf/RAG Project Dataset"
    pdf_paths = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

    # Sample question (you can modify)
    query = "What are the main components of a RAG model and how do they interact?"

    if not pdf_paths:
        print("❌ No PDF files found in './pdfs'. Please add your research papers.")
    else:
        answer, citations = run_rag_system(query, pdf_paths)
        print("\n✅ Query:", query)
        print("\n📘 Answer:\n", answer)
        print("\n📌 Citations:")
        for cite in citations:
            print(cite)


Processing: /content/drive/MyDrive/RAG Project Dataset pdf/RAG Project Dataset/2005.14165v4.pdf
Processing: /content/drive/MyDrive/RAG Project Dataset pdf/RAG Project Dataset/1706.03762v7.pdf
Processing: /content/drive/MyDrive/RAG Project Dataset pdf/RAG Project Dataset/2005.11401v4.pdf


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



✅ Query: What are the main components of a RAG model and how do they interact?

📘 Answer:
 Question: What are the main components of a RAG model and how do they interact?
Context: retriever p(zjx)with parameters that returns (top-K truncated) distributions over text passages given a query xand (ii) a generator p(yijx;z;y 1:i 1)parametrized 1Code to run experiments with RAG has been open-sourced as part of the HuggingFace Transform- ers Library [ 66] and can be found at https://github.com/huggingface/transformers/blob/master/ examples/rag/ . An interactive demo of RAG models can be found at https://huggingface.co/rag/ 2 bythat generates a current token based on a context of the previous i 1tokensy1:i 1, the original inputxand a retrieved passage z. To train the retriever and generator end-to-end, we treat the retrieved document as a latent variable. We propose two models that marginalize over the latent documents in different ways to produce a distribution over generated text. In o